# 一、代码

In [56]:
import time
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
def chinese_word_cut(mytext):
    return ' '.join(jieba.cut(mytext))
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file, encoding='utf-8') as f:
        stopwords = f.read( )
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print('Topic #%d:' % topic_idx)
        print(' '.join([feature_names[i]
                      for i in topic.argsort()[:-n_top_words -1:-1]]))
    print()
df = pd.read_csv('data_20180514_20180515.csv', encoding='gb18030')
df.columns = ['title', 'author', 'comment'] 
X = df[['comment']]
X['cutted_comment'] = X.comment.apply(chinese_word_cut)
stop_words_file = 'mystopwords.txt'
stopwords = get_custom_stopwords(stop_words_file)
vect = CountVectorizer()
term_matrix = pd.DataFrame(vect.fit_transform(X.cutted_comment).toarray(), columns=vect.get_feature_names())
vect = CountVectorizer(stop_words=frozenset(stopwords)) #加上停用词去除功能
term_matrix = pd.DataFrame(vect.fit_transform(X.cutted_comment).toarray(), columns=vect.get_feature_names())
max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 3 # 在地狱这一数量的文档中出现的关键词（过于独特），去除掉。
vect = CountVectorizer(max_df=max_df,
                      min_df=min_df,
                      token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                      stop_words=frozenset(stopwords))
term_matrix = pd.DataFrame(vect.fit_transform(X.cutted_comment).toarray(), columns=vect.get_feature_names())
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                               max_features=n_features,
                               max_df=max_df,
                               min_df=min_df,
                               token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                               stop_words=frozenset(stopwords))
tf = tf_vectorizer.fit_transform(X.cutted_comment)
n_topics = 5
lda = LatentDirichletAllocation(n_topics=n_topics,
                               max_iter=50,
                               learning_method='online',
                               learning_offset=50.,
                               random_state=0)
lda.fit(tf)
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic #0:
股份 富士康 发行 披露 投资者 战略 年度 投资 工业 科技 公告 信息 金额 市场 资金 上市公司 上市 项目 证券 配售
Topic #1:
银行 同比 增长 减少 企业 增加 净利润 员工 项目 增速 营收 调控 下降 业务 股份 城市 百分点 上市 资产 房地产
Topic #2:
企业 流动性 货币政策 资金 降准 mlf 中国 融资 外汇 我国 央行 风险 金融 经济 人民币 煤炭 金融机构 市场 上市公司 工具
Topic #3:
个股 基金 市场 购买 机构 资金 调研 板块 业绩 行业 a股 股份 开户 数据 电子 投资 中国 科技 涨幅 证券
Topic #4:
中国 发展 市场 企业 产品 技术 领域 业务 平安 投资 科技 合作 行业 汽车 集团 全球 上海 平台 服务 提升



# 二、5个主题（默认）

In [57]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      26.364643        1       1  0.012010 -0.024087
3      24.246583        1       2 -0.136359  0.108574
0      23.644552        1       3 -0.174919 -0.124152
2      13.407618        1       4  0.201317 -0.101654
1      12.336605        1       5  0.097951  0.141319, topic_info=     Category        Freq  Term       Total  loglift  logprob
term                                                         
385   Default  427.000000   富士康  427.000000  30.0000  30.0000
54    Default  436.000000    个股  436.000000  29.0000  29.0000
952   Default  441.000000    银行  441.000000  28.0000  28.0000
294   Default  391.000000    同比  391.000000  27.0000  27.0000
344   Default  466.000000    增长  466.000000  26.0000  26.0000
788   Default  779.000000    股份  779.000000  25.0000  25.0000
277   Default  351.000000    发行  351.000000  24.0000  24.0000
651   Default  176.000000   流动性  176.000000  23.0000  23.0000
876   Default  168.000000  货币政策  168.000000  22.0000  22.0000
335   Default  380.000000    基金  380.000000  21.0000  21.0000
122   Default  638.000000    企业  638.000000  20.0000  20.0000
879   Default  288.000000    购买  288.000000  19.0000  19.0000
207   Default  189.000000    减少  189.000000  18.0000  18.0000
864   Default  228.000000    调研  228.000000  17.0000  17.0000
495   Default  308.000000    披露  308.000000  16.0000  16.0000
964   Default  121.000000    降准  121.000000  15.0000  15.0000
3     Default  117.000000   mlf  117.000000  14.0000  14.0000
421   Default  214.000000    年度  214.000000  13.0000  13.0000
472   Default  279.000000    战略  279.000000  12.0000  12.0000
437   Default  157.000000    开户  157.000000  11.0000  11.0000
347   Default  100.000000    外汇  100.000000  10.0000  10.0000
891   Default  519.000000    资金  519.000000   9.0000   9.0000
491   Default  351.000000   投资者  351.000000   8.0000   8.0000
357   Default   95.000000    央行   95.000000   7.0000   7.0000
337   Default  215.000000    增加  215.000000   6.0000   6.0000
613   Default  283.000000    板块  283.000000   5.0000   5.0000
935   Default  131.000000    配售  131.000000   4.0000   4.0000
951   Default  171.000000    金额  171.000000   3.0000   3.0000
820   Default  147.000000    融资  147.000000   2.0000   2.0000
299   Default  103.000000    员工  103.000000   1.0000   1.0000
...       ...         ...   ...         ...      ...      ...
532    Topic5   27.274177    支出   34.792724   1.8491  -5.7041
16     Topic5   22.400881    万科   28.731847   1.8437  -5.9010
96     Topic5   38.016316    五大   49.264485   1.8334  -5.3721
687    Topic5   43.223221    环比   56.804938   1.8194  -5.2437
952    Topic5  274.177854    银行  441.700730   1.6157  -3.3963
294    Topic5  242.520427    同比  391.707585   1.6132  -3.5190
934    Topic5   41.626007    部门   56.950034   1.7792  -5.2813
709    Topic5   64.220921   百分点   95.195801   1.6990  -4.8477
344    Topic5  232.608272    增长  466.090882   1.3976  -3.5607
38     Topic5   70.860028    下降  117.100409   1.5903  -4.7494
15     Topic5   40.008177   万亿元   57.043935   1.7379  -5.3210
337    Topic5   97.031568    增加  215.184305   1.2961  -4.4350
342    Topic5   79.481676    增速  168.782916   1.3395  -4.6345
332    Topic5   65.741925    城市  135.830726   1.3669  -4.8243
204    Topic5   94.650636   净利润  254.916638   1.1019  -4.4599
246    Topic5   41.993595    化学   69.613635   1.5872  -5.2726
974    Topic5   87.815284    项目  271.085028   0.9654  -4.5348
122    Topic5  131.432925    企业  638.551834   0.5119  -4.1316
885    Topic5   62.543236    资产  178.556023   1.0436  -4.8742
22     Topic5   45.060261    上升   81.788929   1.4965  -5.2021
534    Topic5   55.263947    收入  148.020344   1.1074  -4.9979
47     Topic5   70.212532    业务  352.004202   0.4805  -4.7585
26     Topic5   63.741986    上市  269.377165   0.6513  -4.8552
546    Topic5   49.080490    政策  146.124839   1.0016  -5.1166
788    Topic5   67.819045    股份  779.272786  

# 三、以下为10个主题
## 其中主题数可以自定义

In [58]:
n_topics = 10    #此处的10可以自定义
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)
print_top_words(lda, tf_feature_names, n_top_words)
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

D:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic #0:
富士康 发行 股份 战略 投资者 年度 工业 市场 投资 公告 上市 配售 项目 召开 股东大会 披露 ipo 锁定 金额 资金
Topic #1:
银行 同比 增长 减少 营收 调控 增速 股份 净利润 百分点 楼市 城市 住房 房地产 下降 资产 增加 环比 上市 政策
Topic #2:
流动性 货币政策 企业 降准 mlf 资金 外汇 央行 增加 金融 我国 银行 工具 融资 人民币 存款 经济 金融机构 货币 利率
Topic #3:
个股 基金 股份 调研 购买 科技 机构 电子 业绩 数据 开户 证券日报 私募 市场 同比 净利润 增长 独角兽 涨幅 概念股
Topic #4:
平安 科技 智慧 区块 金融 技术 平台 中国 服务 合作 医疗 管理 城市 智能 数据 客户 应用 业务 战略 创新
Topic #5:
企业 中国 发展 行业 市场 产能 增长 业务 煤炭 国内 产品 项目 经济 集团 领域 投资 国家 汽车 交易 持续
Topic #6:
中国 资金 上市公司 披露 外资 上海 信息 qfii 投资者 监管 银行 持股 投资 a股 券商 证券 市场 机构 购买 泰国
Topic #7:
投资 成长 当时 过程 周刊 管理 pe 资本 团队 中国 杠杆 估值 利率 上市 企业 银行业 理念 股价 投资者 比较
Topic #8:
解禁 股份 市值 持有 富联 苹果 沪市 工业 控股 本周 股东 深市 亿股 股权 融资 a股 管理 净利润 科技 合并
Topic #9:
板块 市场 a股 资金 估值 证券 指数 行业 创业板 msci 调整 个股 涨幅 行情 预期 医药 反弹 震荡 相对 概念



PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      27.459628        1       1 -0.030706  0.051062
0      15.995003        1       2  0.150945  0.097663
6      15.015054        1       3  0.021993 -0.042747
9      13.498722        1       4  0.016825 -0.151961
3       9.830362        1       5  0.170469 -0.166955
2       6.545211        1       6 -0.232257 -0.006927
1       6.001669        1       7 -0.184712 -0.125161
4       3.556069        1       8 -0.049819  0.213357
7       1.156542        1       9 -0.021285  0.090030
8       0.941742        1      10  0.158548  0.041639, topic_info=     Category        Freq Term       Total  loglift  logprob
term                                                        
490   Default  502.000000   投资  502.000000  30.0000  30.0000
788   Default  672.000000   股份  672.000000  29.0000  29.0000
734   Default  377.000000   科技  377.000000  28.0000  28.0000
294   Default  370.000000   同比  370.000000  27.0000  27.0000
952   Default  435.000000   银行  435.000000  26.0000  26.0000
418   Default  225.000000   平安  225.000000  25.0000  25.0000
385   Default  432.000000  富士康  432.000000  24.0000  24.0000
344   Default  445.000000   增长  445.000000  23.0000  23.0000
58    Default  714.000000   中国  714.000000  22.0000  22.0000
54    Default  372.000000   个股  372.000000  21.0000  21.0000
401   Default  254.000000   工业  254.000000  20.0000  20.0000
408   Default  159.000000   市值  159.000000  19.0000  19.0000
277   Default  375.000000   发行  375.000000  18.0000  18.0000
122   Default  666.000000   企业  666.000000  17.0000  17.0000
748   Default  223.000000   管理  223.000000  16.0000  16.0000
613   Default  380.000000   板块  380.000000  15.0000  15.0000
504   Default  141.000000   持有  141.000000  14.0000  14.0000
129   Default  238.000000   估值  238.000000  13.0000  13.0000
472   Default  288.000000   战略  288.000000  12.0000  12.0000
0     Default  416.000000   a股  416.000000  11.0000  11.0000
491   Default  387.000000  投资者  387.000000  10.0000  10.0000
335   Default  316.000000   基金  316.000000   9.0000   9.0000
864   Default  170.000000   调研  170.000000   8.0000   8.0000
204   Default  248.000000  净利润  248.000000   7.0000   7.0000
488   Default  186.000000   技术  186.000000   6.0000   6.0000
207   Default  182.000000   减少  182.000000   5.0000   5.0000
948   Default  242.000000   金融  242.000000   4.0000   4.0000
879   Default  236.000000   购买  236.000000   3.0000   3.0000
26    Default  287.000000   上市  287.000000   2.0000   2.0000
651   Default  127.000000  流动性  127.000000   1.0000   1.0000
...       ...         ...  ...         ...      ...      ...
19    Topic10    2.330865   三年   21.570809   2.4401  -5.5912
113   Topic10    9.801763   亿股   95.784107   2.3857  -4.1549
615   Topic10    2.599423   构成   25.635900   2.3765  -5.4822
403   Topic10    2.535201   工厂   25.136320   2.3712  -5.5072
869   Topic10    4.488138   财富   46.175478   2.3342  -4.9360
555   Topic10    2.384773   文化   24.653249   2.3294  -5.5684
633   Topic10    2.100560  毛利率   22.344847   2.3008  -5.6953
71    Topic10    2.724522   主体   30.403054   2.2529  -5.4352
483   Topic10    2.805761   执行   31.589036   2.2441  -5.4058
501   Topic10    3.669224  招股书   46.696019   2.1215  -5.1375
868   Topic10    1.749040   贡献   22.643616   2.1044  -5.8784
785   Topic10    9.992360   股东  129.912163   2.1002  -4.1357
806   Topic10    2.906639   药业   39.107274   2.0659  -5.3705
866   Topic10    1.954908   负责   26.541483   2.0568  -5.7671
478   Topic10    3.513368   手机   47.868506   2.0533  -5.1809
516   Topic10   10.614652   控股  146.560481   2.0400  -4.0752
788   Topic10   38.191600   股份  672.363131   1.7970  -2.7949
289   Topic10    6.451524   合计   95.567388   1.9697  -4.5732
820   Topic10    8.837056   融资  142.470450   1.8850  -4.2585
401   Topic10   12.721765   工业  254.248164   1.6702  -3.8942
631   Topic10    5.392168   比例   94.909679   1.7972  -4.7525
845   Topic10  